<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/ExemploRespondendoPerguntaTextoLongo_Llama2_Langchain_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Respondendo Perguntas sobre Textos Longos usando Llama v2.0, Longchain, Chroma e Transformers by HuggingFace

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

**Lista dos modelos:**

https://huggingface.co/models


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

In [ ]:
!pip install pypdf==3.16.4

Bibliota de dependência para manipular os embeddings pelo Langchain.

https://pypi.org/project/sentence-transformers/

In [ ]:
!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

In [ ]:
!pip install chromadb==0.4.15

Biblioteca para realizar a divisão por token.

In [ ]:
!pip install tiktoken==0.5.1

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

In [ ]:
!pip install langchain==0.0.323

Dependências do xformers

https://pypi.org/project/lmdb/

In [ ]:
!pip install lmdb==1.4.1
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 torchtext==0.15.2+cpu torchdata==0.6.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.22.post7 requires torch==2.1.0, but you have torch 2.0.1+cu118 which is incompatible.


Permite maior velocidade e menor consumo de memória nos transformers.

https://pypi.org/project/xformers/

In [ ]:
!pip install xformers==0.0.22.post7

  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchtext 0.15.2+cpu requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have to

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

https://pypi.org/project/bitsandbytes/

In [ ]:
!pip install bitsandbytes==0.41.1

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.24.0

Instala a interface pytorch para o BERT by Hugging Face.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
# Instala a última versão da biblioteca
# !pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

Instala o cliente do huggingface hub para realizar o login.

In [ ]:
!pip install huggingface_hub==0.18.0

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.

In [ ]:
from huggingface_hub.hf_api import HfFolder

ACCESS_TOKEN  = 'COLOQUE O TOKEN DE ACESSO AQUI'

HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## 2.2 - Nome do LLM

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
#nome_modelo = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

#nome_modelo = "meta-llama/Llama-2-13b-hf"
#nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador do LLM

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


## 2.4 - Carregando o Modelo LLM

Carregando o **modelo** da comunidade Huggingface.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import AutoModelForCausalLM
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True,   # Carrega de um repositório confiável
                                             load_in_8bit=True,
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tempo de carregamento do modelo LLM:  0:01:16 (h:mm:ss)


In [ ]:
print(model.config.max_position_embeddings)

4096


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}



## 2.5 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    max_length=2048,
    #top_k=3,
    # num_return_sequences=1,
    # max_new_tokens = 10 # Limita a quantidade de tokens gerada
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.1})

In [ ]:
print(model_llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': {'temperature': 0.1}, 'pipeline_kwargs': None}


## 2.6 - Nome do LM BERT

In [ ]:
#nome_modelo_bert = "bert-large-cased"
#nome_modelo_bert = "bert-base-cased"
nome_modelo_bert = "neuralmind/bert-large-portuguese-cased"
#nome_modelo_bert = "neuralmind/bert-base-portuguese-cased"

## 2.7 - Carregando o LM BERT

A classe HuggingFaceBgeEmbeddings realiza o download do BERT via HuggingFace.

Os modelos [BGE](https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface) no HuggingFace são os melhores modelos de embeddings de código aberto. O modelo BGE é criado pela Academia de Inteligência Artificial de Pequim(*Beijing Academy of Artificial Intelligence*-BAAI) . BAAI é uma organização privada sem fins lucrativos envolvida em pesquisa e desenvolvimento de IA.

In [ ]:
# Import das bibliotecas
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carrega os embeddings diretamente do HuggingFace
model_lm = HuggingFaceBgeEmbeddings(
    model_name=nome_modelo_bert,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Tempo de carregamento do modelo LM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de carregamento do modelo LM:  0:00:14 (h:mm:ss)


In [ ]:
print(model_lm)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) model_name='neuralmind/bert-large-portuguese-cased' cache_folder=None model_kwargs={'device': 'cuda'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: '


Carrega o tokenizador do bert

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo_bert + ' da comunidade...')

tokenizer_bert = AutoTokenizer.from_pretrained(nome_modelo_bert)

Carregando o tokenizador neuralmind/bert-large-portuguese-cased da comunidade...


# 3 - Pergunta em texto longo

Artigos que auxiliaram a criar o recuperador de texto de perguntas em texto longo.

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

## 3.1 - Carrega os Documentos

Carrega de PDF ou HTML.

Escolha uma das formas de carregar os dados.

Existem outros formas de carregamento.

https://python.langchain.com/docs/modules/data_connection/document_loaders/

### 3.1.1 - Carrega os documentos de PDF

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

####  Download do PDF


Download do PDF do livro "As Vítimas Algozes".

PDF: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977


Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070


In [ ]:
# Import das bibliotecas
# import subprocess

# # As Vítimas Algozes
# urlpdf = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# destino = 'arquivo1.pdf'

# # Executa o comando wget no prompt
# subprocess.call(["wget", urlpdf, "-O", destino])

#### Carrega o PDF



In [ ]:
# # Import das bibliotecas
# from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# # Define o diretório
# diretorio = '/content'

# # Cria o carregar dos documentos do diretório
# # Pode ser usado o PyPDFLoader para um arquivo
# carregador = PyPDFDirectoryLoader(diretorio)

# # Carrega os documentos
# documentos = carregador.load()

### 3.1.2 - Carrega os documentos da WEB

https://python.langchain.com/docs/integrations/document_loaders/web_base

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

#### Carrega o HTML

Carrega o HTML do livro "As Vítimas Algozes".

HTML: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979

Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070



In [ ]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

urlhtml = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# Cria o carregador da página
carregador = WebBaseLoader(urlhtml)

# Carrega os documentos
documentos = carregador.load()

Exibe parte dos dados carregados do PDF ou HTML.

In [ ]:
print("Quantidade de documentos(páginas):", len(documentos))
pagina = 0
print("Trecho página(", pagina, ") :", documentos[pagina].page_content[0:500])
print()
print("Medadados:", documentos[0].metadata)

Quantidade de documentos(páginas): 1
Trecho página( 0 ) : 















As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma ven

Medadados: {'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'title': 'As vítimas algozes - Joaquim Manuel de Macedo', 'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.'}


## 3.2 - Divide e sobrepõe os documentos em chuks

Carrega o documentos e realiza o divisão do documento em pedaços(chunks) e faz a sobreposição(overlap) para garantir o contexto semântico entre os pedaços.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto no link: https://chunkerizer.streamlit.app/

In [ ]:
# Import das bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import time

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    #length_function = len, # Usa o comprimento dos caracteres do texto como medida de tamanho
    length_function = lambda x: len(tokenizer_bert.tokenize(x)), # Usa a quantidade de tokens gerados pelo tokenizador do BERT como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos documentos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos html em {tempo_final - tempo_inicio} segundos!")
print(f"Quantidade de chunks: {len(chunks)}")

Carregando e dividindo 1 documentos html em 5.110030889511108 segundos!
Quantidade de chunks: 375


Mostra alguns chunks. A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [ ]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    print('chunk #',i, ' qtde char:', len(chunk.page_content),' qtde token:', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char: 217  qtde token: 25  qtde token bert: 50  start_index: 16

As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1  qtde char: 1922  qtde token: 308  qtde token bert: 497  start_index: 235

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do traba

In [ ]:
maior_chunk_token = 0
maior_chunk_token_bert = 0
maior_chunk_character = 0

for i, chunk in enumerate(chunks):
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens),' qtde token bert:', len(tokens_bert) )

    # Procura os maiores valores
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(tokens_bert) > maior_chunk_token_bert:
      maior_chunk_token_bert = len(tokens_bert)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk token bert:", maior_chunk_token_bert)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 217  qtde token : 25  qtde token bert: 50
chunk # 1  qtde char : 1922  qtde token : 308  qtde token bert: 497
chunk # 2  qtde char : 1956  qtde token : 316  qtde token bert: 494
chunk # 3  qtde char : 1972  qtde token : 301  qtde token bert: 495
chunk # 4  qtde char : 1907  qtde token : 299  qtde token bert: 483
chunk # 5  qtde char : 1907  qtde token : 297  qtde token bert: 492
chunk # 6  qtde char : 1873  qtde token : 283  qtde token bert: 490
chunk # 7  qtde char : 1934  qtde token : 302  qtde token bert: 481
chunk # 8  qtde char : 1915  qtde token : 298  qtde token bert: 491
chunk # 9  qtde char : 2139  qtde token : 321  qtde token bert: 500
chunk # 10  qtde char : 2048  qtde token : 312  qtde token bert: 491
chunk # 11  qtde char : 2072  qtde token : 317  qtde token bert: 500
chunk # 12  qtde char : 2073  qtde token : 318  qtde token bert: 497
chunk # 13  qtde char : 1936  qtde token : 290  qtde token bert: 499
chunk # 14  qtde char : 2009  qtde token : 290 

## 3.3 - Armazena os chunks

Utiliza o Chroma um banco de dados de embeddings de código aberto e leve.

https://www.trychroma.com/

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

In [ ]:
# Import das bibliotecas
from langchain.vectorstores import Chroma

# Armazena os chunks usando o Chroma
vectorstore = Chroma.from_documents(documents=chunks, embedding=model_lm, persist_directory="./bancodedados", )

## 3.4 - Função de envio de perguntas ao LLM

In [ ]:
# Import das bibliotecas
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import torch
import gc

def avaliarContexto(texto):

  # Cria o texto de prompt
  prompt_template = """Você é um assistente de IA útil e fornece a resposta para a pergunta com base no contexto fornecido.
Contexto: {context}
>>PERGUNTA<< {question}
>>RESPOSTA<<"""

  # Cria o prompt
  # As variáveis "context" e "question" não podem ser alteradas
  # O parâmetro "context" recebe o contexto do carregado e armazenado no Chroma
  # O parâmetro "question" recebe a pergunta realizada sobre o contexto.
  prompt = PromptTemplate(input_variables=["context", "question"],
                          template=prompt_template)

  # Instancia o chain
  chain_type_kwargs = {"prompt": prompt}
  chain = RetrievalQA.from_chain_type(
        llm=model_llm,
        chain_type="stuff",
        #retriever=vectorstore.as_retriever(), # Passa o contexto(livro carregado)
        retriever=vectorstore.as_retriever(search_kwargs={'k':1}),
        chain_type_kwargs=chain_type_kwargs
    )

  # Executa o prompt no llm
  resultado = chain.run(texto)

  # Esvazia a memória
  del chain
  gc.collect()
  torch.cuda.empty_cache()

  return resultado

## 3.5 - Algumas perguntas usando o contexto

Guarda o tempo de início de realização das perguntas

In [ ]:
# Importando as bibliotecas do Modelo
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

In [ ]:
texto = "Qual o período que ocorre a história do texto?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 O período que ocorre na história do texto é o século XIX.


In [ ]:
texto = "Considerando o contexto em qual período que ocorre os fatos?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 O contexto é o século XIX, no Brasil, durante a época da escravidão.


In [ ]:
texto = "Qual o nome de todos os personagens do contexto?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Simeão, o Barbudo, Eufêmia e outros cúmplices.


In [ ]:
texto = "Considerando o contexto qual o nome de todos os personagens da história?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Os personagens da história são:

1. Simeão
2. Eufêmia
3. o Barbudo
4. os cúmplices (não especificados)

Portanto, o nom
e de todos os personagens da história é:

1. Simeão
2. Eufêmia
3. o Barbudo
4. os cúmplices (não especificados)


In [ ]:
texto = "Quem são os principais personagens do contexto?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Os principais personagens do contexto são:

1. Joaquim Manuel de Macedo - autor do livro "As Vítimas-Algozes"
2. Ana - 
personagem principal do livro, que é uma das vítimas-algozes
3. Carlos - outra vítima-algoze, que é amigo de Ana
4. Mari
a - mãe de Ana, que é uma das principais figuras da história
5. Jorge - um dos principais suspeitos da morte de Ana e da
s outras vítimas-algozes.


In [ ]:
texto = "Quais os nomes dos principais personagens do contexto?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Os principais personagens do contexto são:

1. Joaquim Manuel de Macedo - autor do livro
2. Ana - personagem principal 
do livro
3. Carlos - marido de Ana
4. Maria - mãe de Ana
5. Antônio - amigo de Ana
6. Leonor - amiga de Ana
7. Dr. Costa
 - médico que atende Ana
8. Sr. Ferreira - proprietário da fazenda onde Ana morava.


In [ ]:
texto = "Considerando o contexto quem são os principais personagens da história?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Os principais personagens da história são:

1. Simeão: é o principal personagem da história, que tenta fugir com o ouro
 e a prata, mas é capturado e enforcado.
2. Barbudo: é um dos cúmplices de Simeão, que também tenta fugir, mas é captura
do e enforcado.
3. Eufêmia: é outro cúmplice de Simeão, que também tenta fugir, mas é capturada e enforcada.
4. Carrasco
: é o responsável pela aplicação da lei e pela execução dos criminosos. Ele é o narrador da história e descreve os event
os que ocorreram.


In [ ]:
texto = "Considerando o contexto qual o nome dos principais personagens da história?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Os principais personagens da história são:

1. Joaquim Manuel de Macedo - o autor da obra e o narrador da história.
2. 
Ana - a mulher que é objeto de desejo e violência por parte dos personagens masculinos.
3. O rapaz - um dos principais p
ersonagens masculinos que participa da história e é responsável pela violência contra Ana.
4. O outro rapaz - outro pers
onagem masculino que também participa da história e é responsável por outra forma de violência contra Ana.
5. O pai - o 
pai de Ana, que é descrito como um homem cruel e violento.
6. A mãe - a mãe de Ana, que é descrita como uma mulher fraca
 e submissa.
7. O juiz - um personagem que aparece na história e é responsável por julgar o caso de violência contra Ana
.
8. O advogado - um personagem que representa Ana e busca provar a culpa dos principais personagens masculinos envolvid
os na história.

Esses são os principais personagens da história, e eles são todos envolvidos na trama de violência e ma
nipulação que se desenvolve na o

In [ ]:
texto = "Lista para mim as palavras que possuem relação com escravidão. Quantifique as ocorrências no texto"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 As palavras que possuem relação com escravidão no texto são:

1. Escravidão (mencionada em 3 ocasiões)
2. Escravo (menc
ionado em 2 ocasiões)
3. Mão (mencionada em 2 ocasiões)
4. Trabalho (mencionado em 2 ocasiões)
5. Condenação (mencionada
 em 1 oportunidade)
6. Castigo (mencionado em 1 oportunidade)
7. Opressão (mencionada em 1 oportunidade)
8. Privilégio (
mencionado em 1 oportunidade)

Observação: As ocorrências de cada palavra podem variar dependendo do contexto e da inter
pretação do texto.


In [ ]:
texto = "Considerando o contexto que palavras possuem relação com escravidão?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 As palavras "vítimas-algozes" de Joaquim Manuel de Macedo possuem relação com a escravidão, pois o autor usou essa expr
essão para descrever as pessoas que foram submetidas à escravidão. A palavra "vítima" indica que essas pessoas foram vít
imas de uma situação de oppressão e exploração, enquanto "algozes" é um termo que evoca a ideia de algo que é estranho o
u desconhecido, o que reflete a ideia de que a escravidão era uma realidade estranha e desconhecida para muitas pessoas 
na época. Além disso, a expressão "vítimas-algozes" também pode ser entendida como uma crítica à ideologia da escravidão
, que considerava as pessoas escravas como se fossem objetos ou propriedade, em vez de seres humanos com direitos e dign
idade.


In [ ]:
texto = "Considerando o contexto que palavras possuem relação semântica com escravidão?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras "vítimas-algozes" possuem relação semântica com a escravidão, pois elas são usadas para descrever as pessoa
s que são tratadas como bens ou objetos, sem direitos ou liberdades. A palavra "vítima" é usada para descrever uma pesso
a que sofre um mal ou injustiça, enquanto a palavra "algoze" é usada para descrever algo que é considerado como uma merc
adoria ou um bens. Juntas, as palavras "vítimas-algozes" são usadas para descrever as pessoas que são tratadas como bens
 ou objetos, sem direitos ou liberdades, o que é semelhante à escravidão.


In [ ]:
texto = "Considerando o contexto que palavras estão relacionadas a escravidão mas deixa isto implicito?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Sim, é possível notar que as palavras relacionadas a escravidão estão presentes no contexto, mas não são explicitamente
 mencionadas. Ao descrever a situação dos personagens, a obra sugere que a escravidão é um tema importante e que está re
lacionada à história, mas não é explicitamente mencionada. Isso pode ser visto como uma forma de criticar a escravidão d
e forma implicita, sem necessariamente mencioná-la diretamente.


Textos do trabalho do **Leandro da Silveira Dias**

In [ ]:
texto = "Considerando a palavra \"negro\" com alvo, identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Em relação ao contexto fornecido, as palavras semanticamente similares ao termo "negro" são:

* Preto
* Escravo
* Crimi
nal
* Malvado
* Perverso

Essas palavras têm o mesmo contexto de uso e conotação negativa que o termo "negro" em relação
 às víctimas do crime. É importante notar que o uso dessas palavras pode ser considerado ofensivo e discriminatório, e d
eve ser evitado em favor de termos mais neutros e respectuosos.


In [ ]:
texto = "Considerando a palavra alvo \"negro\", identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Ao considerar a palavra "negro" no contexto da obra "As Vítimas-Algozes", de Joaquim Manuel de Macedo, identificamos ou
tras palavras semanticamente similares, que são:

1. Preto: É a palavra mais utilizada para descrever a cor do pele em P
ortugal, e é utilizada também em outros países lusófonos.
2. Africano: Refere-se a uma pessoa que é originária da África
, independentemente da cor da pele.
3. Afro-brasileiro: É um termo utilizado para descrever uma pessoa que tem ancestral
idade africana e que nasceu ou reside no Brasil.
4. Afro-latino: É um termo utilizado para descrever uma pessoa que tem 
ancestralidade africana e que nasceu ou reside em um país lusófono.
5. Afro-descendente: É um termo utilizado para descr
ever uma pessoa que tem ancestralidade africana e que nasceu ou reside em um país lusófono.

É importante notar que a ut
ilização dessas palavras pode variar dependendo do contexto e do foco da discussão. É recomendável utilizar palavras que
 são menos discriminatórias e ma

In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra \'negro\'"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  As palavras 'pundonor','vergonha', 'escravidão', 'negro' estão ligadas entre si na sentença, pois todas elas têm um re
lacionamento com a cor da pele e com a condição social do negro. 'Pundonor' e'vergonha' são termos que expressam a nobre
za e a dignidade, enquanto 'escravidão' é um termo que designa a condição de escravo, que era comum na época e que envol
via a propriedade de um ser humano por outro. 'Negro' é um termo que designa a cor da pele dos afro-americanos, e que ta
mbém foi utilizado para designar a condição social e a discriminação que esses grupos enfrentavam.


In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra alvo \'negro\' considerando o contexto."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Na sentença 'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da v
ingança.' as palavras 'rancor' e 'desejo' estão ligadas à palavra 'negro' no contexto, pois o escravo tem um sentimento 
negativo contra os seus cúmplices e proprietários, que o tratam com crueldão e injustiça.


In [ ]:
texto = "Usando leitura distante realize a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Com base na leitura distante realizada, as palavras que possuem ligação com a palavra alvo 'negro' na sentença 'Em falt
a de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.' são:

1. fal
ta
2. de
3. negro
4. comporta
5. escravidão
6. negro

Essas palavras possuem uma conexão semântica com a palavra alvo 'n
egro', pois estão relacionadas ao tema de escravidão e raça negra.


In [ ]:
texto = "Considerando o contexto use leitura distante para realizar a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Considerando o contexto, as palavras que possuem ligação com a palavra alvo 'negro' são:

1. escravidão
2. pundonor
3. 
vergonha
4. rancor
5. vingança

Essas palavras possuem uma conexão semântica com a palavra 'negro' pois estão relacionad
as ao tema da escravidão e da discriminação racial, que é mencionada na sentença.


In [ ]:
texto = "Usando leitura distante realize a analise da sentença: \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ e liste as palavras possuem relação semântica com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras que possuem relação semântica com a palavra alvo 'negro' na sentença analisada são:

* rancor
* desejo
* vi
ngança
* escravidão
* pundonor
* vergonha

Essas palavras compartilham um significado relacionado ao tema da escravidão 
e à cor do corpo humano, o que é relevante para a interpretação da sentença.


Tempo final de execução das perguntas

In [ ]:
print("Tempo de execução das perguntas:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de execução das perguntas:  0:11:47 (h:mm:ss)
